In [2]:
from backtesting import set_bokeh_output
set_bokeh_output(notebook=False)

# !python3 -m pip install bokeh==2.4.3

In [3]:
import sys
from datetime import timedelta

In [4]:
# sys.path.insert(1, '/home/kenny/algotrading/model_training')
sys.path.insert(1, "/Users/arsenchik/Desktop/study/dipploma/machine_learning_in_hft/algotrading/src/backtesting")
sys.path.insert(1, '/Users/arsenchik/Desktop/study/dipploma/machine_learning_in_hft/algotrading/src/models')
sys.path.insert(1, '/Users/arsenchik/Desktop/study/dipploma/machine_learning_in_hft/algotrading/src/configs')
sys.path.insert(1, "/Users/arsenchik/Desktop/study/dipploma/machine_learning_in_hft/algotrading/src/data")

In [5]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from tqdm import tqdm
from backtesting import Backtest, Strategy
from sqlalchemy import create_engine, text
import pandas_ta as pta
from backtesting.lib import crossover

In [6]:
from strategies import MultipleBasic, SmaCross, BinaryBasic, Multiple, Binary, Random
from config import *
from dataset import Data
from experiments import EXPERIMENT_ID_TO_FEATURES

In [7]:
url = DATABASE_URL_15MINUTE
td_days = 100

In [8]:
engine = create_engine(url=url)

with engine.connect() as connection:
        # query = "SELECT name FROM sqlite_schema WHERE type='table'"
        # symbols = [item[0] for item in connection.execute(text(query)).fetchall()]
        # for symbol in symbols:
        symbol = 'BTCUSDT'
        qry = f"SELECT * FROM '{symbol}' WHERE Open_Time < '{pd.to_datetime('today') - timedelta(days = td_days)}'"
        data = pd.DataFrame(connection.execute(text(qry)))
        data['Open_Time'] = pd.to_datetime(data['Open_Time'])
        data.set_index("Open_Time", inplace=True)
        # data.index = pd.to_datetime(data.index)
        # data.drop(['Close_Time'], axis=1, inplace=True)
connection.close()

In [9]:
data

,Open,High,Low,Close,Volume
Open_Time,,,,,
2023-02-22 06:15:00,24090.10,24097.13,24021.33,24040.45,2901.37593
2023-02-22 06:30:00,24039.45,24049.31,23951.57,23988.82,4538.32091
2023-02-22 06:45:00,23988.82,24048.62,23931.09,23950.15,4327.60748
2023-02-22 07:00:00,23950.15,24024.39,23931.00,24003.58,3940.67783
2023-02-22 07:15:00,24004.90,24088.98,23981.59,23981.59,3709.70038
...,...,...,...,...,...
2023-02-27 05:00:00,23490.81,23509.94,23450.00,23462.73,1896.65355
2023-02-27 05:15:00,23463.64,23487.89,23443.00,23479.82,1635.03583
2023-02-27 05:30:00,23479.47,23484.59,23450.00,23455.37,1561.69089


In [10]:
data[data.index < data.index[0] + pd.Timedelta(days=10)]

,Open,High,Low,Close,Volume
Open_Time,,,,,
2023-02-22 06:15:00,24090.10,24097.13,24021.33,24040.45,2901.37593
2023-02-22 06:30:00,24039.45,24049.31,23951.57,23988.82,4538.32091
2023-02-22 06:45:00,23988.82,24048.62,23931.09,23950.15,4327.60748
2023-02-22 07:00:00,23950.15,24024.39,23931.00,24003.58,3940.67783
2023-02-22 07:15:00,24004.90,24088.98,23981.59,23981.59,3709.70038
...,...,...,...,...,...
2023-02-27 05:00:00,23490.81,23509.94,23450.00,23462.73,1896.65355
2023-02-27 05:15:00,23463.64,23487.89,23443.00,23479.82,1635.03583
2023-02-27 05:30:00,23479.47,23484.59,23450.00,23455.37,1561.69089


In [11]:
import lightgbm as lgb
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score

def _slice_data(df: pd.DataFrame, timeframe: int, index_id: int) -> pd.DataFrame:
    return df[df.index < df.index[index_id] + pd.Timedelta(days=timeframe)]


def _training(df: pd.DataFrame):
    model = lgb.Class

In [12]:
# data['RSI'] = pta.rsi(data['Close'], length=21)
# data['EMA_6'] = pta.ema(data['Close'], length=6)
# data['EMA_22'] = pta.ema(data['Close'], length=22)
# data['EMA_127'] = pta.ema(data['Close'], length=127)

In [13]:
def _get_ema(close, index, length):
    df = pd.DataFrame({"data": list(index), "close": list(close)}).set_index("data")
    return list(pta.ema(df.close, length=length))

def _get_rsi(close, index, length):
    df = pd.DataFrame(({"data": list(index), "close": list(close)})).set_index("data")
    return list(pta.rsi(df.close, length=length))

def _get_macd_main_line(close, index, fast, slow, signal):
    df = pd.DataFrame({"data": list(index), "close": list(close)}).set_index("data")
    return list(pta.macd(df.close, fast=fast, slow=slow, signal=signal)["MACD_12_26_9"])

def _get_macd_signal_line(close, index, fast, slow, signal):
    df = pd.DataFrame({"data": list(index), "close": list(close)}).set_index("data")
    return list(pta.macd(df.close, fast=fast, slow=slow, signal=signal)["MACDs_12_26_9"])

def _get_macd_hist(close, index, fast, slow, signal):
    df = pd.DataFrame({"data": list(index), "close": list(close)}).set_index("data")
    return list(pta.macd(df.close, fast=fast, slow=slow, signal=signal)["MACDh_12_26_9"])


## Random classifier

In [14]:
cash = 100000
commission = .0001
bt_random = Backtest(data, Random, cash=cash, commission=commission)
output = bt_random.run()

In [15]:
output

Start                     2023-02-22 06:15:00
End                       2023-02-27 06:00:00
Duration                      4 days 23:45:00
Exposure Time [%]                   99.583333
Equity Final [$]                 98812.726422
Equity Peak [$]                 100457.683143
Return [%]                          -1.187274
Buy & Hold Return [%]                -2.75128
Return (Ann.) [%]                  -54.892111
Volatility (Ann.) [%]                3.576117
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                    -1.94422
Avg. Drawdown [%]                   -0.420898
Max. Drawdown Duration        4 days 03:30:00
Avg. Drawdown Duration        0 days 16:58:00
# Trades                                  244
Win Rate [%]                        40.983607
Best Trade [%]                       1.181789
Worst Trade [%]                      -2.01712
Avg. Trade [%]                    

In [16]:
bt_random.plot()

Row(id='1475', ...)

## Binary LightCoin strategy

In [16]:
binary_model_path_txt = '/Users/arsenchik/Desktop/study/dipploma/machine_learning_in_hft/algotrading/models/binary_model_1_min.txt'

binary_model = lgb.Booster(model_file=binary_model_path_txt)

features = EXPERIMENT_ID_TO_FEATURES[24]
test_data = Data(data)
test_data.create_features()
test_data.df['predicted_proba'] = binary_model.predict(test_data.df[features])

In [17]:
cash = 100000
commission = .0001

bt = Backtest(test_data.df, Binary, cash=cash, commission=commission)
output = bt.run()

In [18]:
output

Start                     2023-02-22 06:15:00
End                       2023-02-27 06:00:00
Duration                      4 days 23:45:00
Exposure Time [%]                       13.75
Equity Final [$]                   99432.5586
Equity Peak [$]                 100054.570674
Return [%]                          -0.567441
Buy & Hold Return [%]                -2.75128
Return (Ann.) [%]                   -9.213614
Volatility (Ann.) [%]                0.775158
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                    -0.75796
Avg. Drawdown [%]                    -0.43436
Max. Drawdown Duration        4 days 21:30:00
Avg. Drawdown Duration        2 days 11:38:00
# Trades                                   11
Win Rate [%]                        36.363636
Best Trade [%]                       0.520845
Worst Trade [%]                     -1.374494
Avg. Trade [%]                    

In [19]:
bt.plot()

Row(id='2679', ...)

## Multiple LigthCoin strategy

In [20]:
cash = 100000
commission = .0001

bt = Backtest(data, Multiple, cash=cash, commission=commission)
output = bt.run()
# stats = bt.optimize(price_delta=[0.002, 0.004, 0.001],
#                   maximize='Sharpe Ratio')

In [21]:
output

Start                     2023-02-22 06:15:00
End                       2023-02-27 06:00:00
Duration                      4 days 23:45:00
Exposure Time [%]                        82.5
Equity Final [$]                 98529.317921
Equity Peak [$]                 100293.930315
Return [%]                          -1.470682
Buy & Hold Return [%]                -2.75128
Return (Ann.) [%]                  -54.014657
Volatility (Ann.) [%]                4.493992
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -1.917481
Avg. Drawdown [%]                   -0.806046
Max. Drawdown Duration        3 days 18:00:00
Avg. Drawdown Duration        1 days 14:05:00
# Trades                                  204
Win Rate [%]                        24.019608
Best Trade [%]                       2.130106
Worst Trade [%]                     -0.862477
Avg. Trade [%]                    

In [23]:
bt.plot()

Row(id='5089', ...)